In [1]:
import ee
import os
import folium
import time
from folium import plugins
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns
import math 
from sklearn.linear_model import TheilSenRegressor
from numpy import nan
import scipy
import statsmodels.api as sm
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
import numpy.polynomial.chebyshev as cheb
from tqdm import tqdm

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
#inputList= ["projects/ee-modis250/assets/validationGBOV/NEON_2013-22_RMs_updated_parsed"]
inputList=["projects/sentinel2downloder/assets/ottawa_site"]
outpath='C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/'

In [4]:
fbasename=os.path.basename(inputList[0])
sampleLAIFileName = os.path.join(outpath,fbasename+'_LAI.pkl')
samplefCOVERFileName = os.path.join(outpath,fbasename+'_fCOVER.pkl')
samplefAPARFileName = os.path.join(outpath,fbasename+'_fAPAR.pkl')
print('LAI file:\n %s\nfCOVER file:\n %s\nfAPAR file:\n %s\n'%(sampleLAIFileName,samplefCOVERFileName,samplefAPARFileName))

LAI file:
 C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/ottawa_site_LAI.pkl
fCOVER file:
 C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/ottawa_site_fCOVER.pkl
fAPAR file:
 C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/ottawa_site_fAPAR.pkl



In [5]:
# LEAFToolbox-SL2P: # Network for producing estimates for SL2P for S2 MSI
def s2_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1')
def s2_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_error') 
# LEAFToolbox-SL2P: # List of coded values corresponding to input domain for S2 MSI
def s2_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/S2_SL2P_WEISS_ORIGINAL_DOMAIN')

# LEAFToolbox-SL2P: # List of estimated values corresponding to output range for S2 MSI
def s2_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/COPERNICUS_S2_SR_SL2P_RANGE')
# LEAFToolbox-SL2P: # Table mapping input partitions to networks for S2 MSI
def s2_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Parameter_file_sl2p')    
# land cover partition, we merge nalcms last so it takes precedence in a mosaic
def s2_createImageCollection_partition():
    return ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global") \
             .map( lambda image: image.select("discrete_classification") \
             .remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")) \
             .merge(ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles') \
             .map(lambda image: image.select("b1").rename("partition")))

# LEAFToolbox-SL2P: # Mapping of input partition to networks for S2 MSI
def s2_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Legend_sl2p')

In [6]:
def l8_createFeatureCollection_estimates():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_OUTPUT')
def l8_createFeatureCollection_errors():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_ERRORS')  
def l8_createFeatureCollection_domains():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_DOMAIN')
def l8_createFeatureCollection_range():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_RANGE')
def l8_createFeatureCollection_Network_Ind():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Parameter_file_sl2p')
# land cover partition, we merge nalcms last so it takes precedence in a mosaic
def l8_createImageCollection_partition():
    return ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global") \
             .map( lambda image: image.select("discrete_classification") \
             .remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("partition")) \
             .merge(ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles') \
             .map(lambda image: image.select("b1").rename("partition")))
def l8_createFeatureCollection_legend():
    return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Legend_sl2p')

In [7]:
# add a 'date' band: number of days since epoch
def addDate(image):
    return image.addBands(ee.Image.constant(ee.Date(image.date()).millis().divide(86400000)).rename('date').toUint16())

In [8]:
# mask pixels that are not clear sky in a S2 MSI image
def s2MaskClear(image) :
    qa = image.select('QA60');
    mask = qa.bitwiseAnd(1<<10).eq(0).And(qa.bitwiseAnd(1<<11).eq(0));
    return image.updateMask(mask)

In [9]:
# add s2 geometry bands scaled by 10000
def addS2Geometry(colOptions,image) :
    return (image.addBands(image.metadata(colOptions["vza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosVZA']))
              .addBands(image.metadata(colOptions["sza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosSZA']))
              .addBands(image.metadata(colOptions["saa"]).subtract(image.metadata(colOptions["saa"])).multiply(3.1415).divide(180).cos().multiply(10000).toInt16().rename(['cosRAA'])));

In [10]:
# sentinel 2 land mask
def s2MaskLand(image) :
    #return image.updateMask(image.select('SCL').gt(0))
    return image.updateMask((image.select('SCL').eq(4)).Or(image.select('SCL').eq(5)))

In [11]:
# returns image with selected bands scaled
def scaleBands(bandList,scaleList,image) :
    bandList = ee.List(bandList)
    scaleList = ee.List(scaleList)
    return image.addBands(srcImg = image.select(bandList).multiply(ee.Image.constant(scaleList)).rename(bandList),overwrite = True)

In [12]:
# Determine if inputs fall in domain of algorithm
# Need to be updated to allow for the domain to vary with partition
def invalidInput(sl2pDomain,bandList,image) :  
    sl2pDomain =  ee.FeatureCollection(sl2pDomain).aggregate_array("DomainCode").sort()
    bandList = ee.List(bandList).slice(3)
    image = ee.Image(image)
    # code image bands into a single band and compare to valid codes to make QC band
    image = image.addBands(image.select(bandList)
                              .multiply(ee.Image.constant(ee.Number(10)))
                              .ceil()
                              .mod(ee.Number(10))
                              .uint8()
                              .multiply(ee.Image.constant(ee.List.sequence(0,bandList.length().subtract(1)).map(lambda value: ee.Number(10).pow(ee.Number(value)))))
                              .reduce("sum")  
                              .remap(sl2pDomain, ee.List.repeat(0, sl2pDomain.length()),1)
                              .rename("QC"))
    return image

In [13]:
# returns image with single band named network id corresponding given 
def makeIndexLayer(image,legend,Network_Ind) :  
    image = ee.Image(image)                        # partition image
    legend = ee.FeatureCollection(legend)          # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind) # legend to convert networks to networkIDs
    #get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover= legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                              .map(lambda propertyValue:  ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [14]:
# Read coefficients of a network from csv EE asset
def getCoefs(netData,ind) :
    return((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

In [15]:
# Parse one row of CSV file for a network into a global variable
# We assume a two hidden layer network with tansig functions but
# allow for variable nodes per layer
def makeNets(feature, M) :  
    feature = ee.List(feature);
    M = ee.Number(M);
    
    # get the requested network and initialize the created network
    netData = ee.Feature(feature.get(M.subtract(1)));
    net = {};
    
    # input slope
    num = ee.Number(6);
    start = num.add(1);
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
    net["inpSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    # input offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["inpOffset"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 1 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h1bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # hidden layer 2 weight
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2wt"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # hidden layer 2 bias
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["h2bi"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))

    # output slope
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outSlope"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
  
    # output offset
    num = end.add(1)
    start = num.add(1)
    end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
    net["outBias"] = ee.List.sequence(start,end).map(lambda ind: getCoefs(netData,ind))
    
    return(ee.Dictionary(net))

In [16]:
COLLECTION_OPTIONS = {
    'COPERNICUS/S2_SR': {
      "name": 'S2',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(s2_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    'LANDSAT/LC08/C01/T1_SR': {
      "name": 'L8',
      "description": 'LANDSAT 8',
      "Cloudcover": 'CLOUD_COVER_LAND',
      "Watercover": 'CLOUD_COVER',
      "sza": 'SOLAR_ZENITH_ANGLE',
      "vza": 'SOLAR_ZENITH_ANGLE',
      "saa": 'SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'SOLAR_AZIMUTH_ANGLE',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(l8_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(l8_createFeatureCollection_errors()),
      "sl2pDomain": ee.FeatureCollection(l8_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(l8_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(l8_createImageCollection_partition()),
      "legend":  ee.FeatureCollection(l8_createFeatureCollection_legend()),
      "numVariables": 7
    }
}
VIS_OPTIONS = {
    "Surface_Reflectance": {
        "COPERNICUS/S2_SR": {
            "Name": 'Surface_Reflectance',
            "description": 'Surface_Reflectance',
            "inp":      [ 'B4', 'B5', 'B6', 'B7', 'B8A','B9','B11','B12']
        }
      },
    "Albedo": {
        "COPERNICUS/S2_SR": {
            "Name": 'Albedo',
            "errorName": 'errorAlbedo',
            "maskName": 'maskAlbedo',
            "description": 'Black sky albedo',
            "variable": 6,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    },
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
      }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      }  
    },
    'CCC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CCC',
            "errorName": 'errorCCC',
            "maskName": 'maskCCC',
            "description": 'Canopy chlorophyll content',
            "variable": 4,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1000]])))
      } 
    },
    'CWC': {
        "COPERNICUS/S2_SR": {
            "Name": 'CWC',
            "errorName": 'errorCWC',
            "maskName": 'maskCWC',
            "description": 'Canopy water content',
            "variable": 5,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[100]])))
      } 
    },
    'DASF': {
        "COPERNICUS/S2_SR": {
            "Name": 'DASF',
            "errorName": 'errorDASF',
            "maskName": 'maskDASF',
            "description": 'Directional area scattering factor',
            "variable": 7,
            "inputBands":      [ 'cosVZA','cosSZA','cosRAA','B3','B4', 'B5', 'B6', 'B7', 'B8A','B11','B12'],
            "inputScaling":      [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
      } 
    }
}

In [17]:
# applies two layer neural network within input and output scaling
def applyNet(outputName,netDict) :   
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))

    # Input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()])) \
                   .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([inp.bandNames()]))
    
    # Hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # Output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
                    .divide(ee.Image(ee.Number(net.get('outSlope')))) 
    
    # Return network output
    return (outputBand.rename(outputName))

In [18]:
def makeNetVars(asset, numNets, variableNum):
    asset = ee.FeatureCollection(asset)
    numNets = ee.Number(numNets)
    variableNum = ee.Number(variableNum)  

    # get selected network 
    filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)
    netVars = ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))
    return netVars

In [155]:
variable = "CCC"
collectionName = "COPERNICUS/S2_SR"
maxCloudcover = 90
filterSize = 0 # length of square filter window applied to product
scaleSize = 20  # scale to produce and sample product
deltaTime = ["2023-08-31","2023-09-01"] # number of days before and after sample
bufferSize = 1

input=inputList[0]
sampleRecords =  ee.FeatureCollection(input).sort('system:time_start', False).map(lambda feature: feature.set('timeStart',feature.get('system:time_start')))
sampleRecords =  sampleRecords.toList(sampleRecords.size())
n=0 
site = ee.Feature(sampleRecords.get(n))
outputName,collectionName,maxCloudcover,bufferSize,outputScale,deltaTime,filterSize=variable,collectionName,maxCloudcover,bufferSize,scaleSize,deltaTime,filterSize

startDate = datetime.strptime(deltaTime[0],"%Y-%m-%d")
endDate =  datetime.strptime(deltaTime[1],"%Y-%m-%d")
sampleFeature = [];
productCollection = []

mapBounds=site.buffer(bufferSize*2).geometry()
colOptions = COLLECTION_OPTIONS[collectionName]
netOptions = VIS_OPTIONS[outputName][collectionName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[collectionName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())


print('Done')

Done


In [156]:
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2P"],numNets,netNum))
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[collectionName]["numVariables"]),1).map(lambda netNum: makeNetVars(COLLECTION_OPTIONS[collectionName]["Collection_SL2Perrors"],numNets,netNum))

In [157]:
len(SL2P.getInfo())

7

In [158]:
input_collection =  ee.ImageCollection(collectionName) \
                      .filterBounds(mapBounds) \
                      .filterDate(startDate, endDate) \
                      .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                      .limit(5000)

#add date band and clip
input_collection = input_collection.map(lambda image: addDate(image)).map(lambda image: image.clip(mapBounds)) 
if collectionName=='COPERNICUS/S2_SR':
    input_collection =  input_collection.map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
elif collectionName=='LANDSAT/LC08/C01/T1_SR': 
    input_collection =  input_collection.map(lambda image: L8MaskClear(image)).map(lambda image: addL8Geometry(colOptions,image))
elif  collectionName=='users/rfernand387/L2avalidation': 
    input_collection = input_collection.map(lambda image: ccrsSentinel2(image)).map(lambda image: s2MaskClear(image)).map(lambda image: addS2Geometry(colOptions,image))
else:
    print('npooooooooooooooooooooooo')

projection = input_collection.first().select(input_collection.first().bandNames().slice(0,1)).projection()
input_collection = input_collection.map( lambda image: image.setDefaultProjection(crs=image.select(image.bandNames().slice(0,1)).projection()).reduceResolution(reducer= ee.Reducer.mean(),maxPixels=1024).reproject(crs=projection,scale=outputScale))
input_collection.size().getInfo()

1

In [159]:
# returns image with single band named networkid corresponding given 
# input partition image remapped to networkIDs
# applies a set of shallow networks to an image based on a provided partition image band
def wrapperNNets(network, partition, netOptions, colOptions, layerName, imageInput) :
    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = netOptions
    colOptions = colOptions
    layerName = layerName
    imageInput = ee.Image(imageInput)

    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])
    #.remap([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,15,15,15,15],0).rename('partition')

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1))); 

    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

    # define list of input names
    return ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
                                                    .map(lambda netIndex: selectNet(imageInput,netList,netOptions.get("inputBands"),netIndex)) \
                                                    .map(lambda netDict: applyNet(layerName,netDict))) \
                                .max() \
                  .addBands(partition) \
                  .addBands(imageInput.select('networkID'))

In [160]:
partition = (COLLECTION_OPTIONS[collectionName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition');
input_collection  =  input_collection.map(lambda image: s2MaskLand(image)) \
                                                .map(lambda image: scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                 .map(lambda image: invalidInput(COLLECTION_OPTIONS[collectionName]["sl2pDomain"],netOptions["inputBands"],image))
input_collection.size().getInfo()

1

In [161]:
def selectNet(image,netList,inputNames,netIndex) :  
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()
    #print(ee.List(netList.get(netIndex)))
    return ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex)))

In [162]:
estimateSL2P =    input_collection.map(lambda image: wrapperNNets(SL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"estimate",image))
uncertaintySL2P = input_collection.map(lambda image: wrapperNNets(errorsSL2P,partition, netOptions, COLLECTION_OPTIONS[collectionName],"error",image))

In [163]:
SR=input_collection.map(lambda image: image.sample(region=site.geometry(), projection=projection, scale=outputScale,geometries=True, dropNulls = False) ).flatten()
E=estimateSL2P.map(lambda image: image.sample(region=site.geometry(), projection=projection, scale=outputScale,geometries=True, dropNulls = False) ).flatten()
U=uncertaintySL2P.map(lambda image: image.sample(region=site.geometry(), projection=projection, scale=outputScale,geometries=True, dropNulls = False) ).flatten()

In [164]:
len(E.getInfo()['features'])

1710

In [165]:
len(SR.getInfo()['features'])

1710

In [166]:
SRd=SR.getInfo()['features']
Ed=E.getInfo()['features']
Ud=U.getInfo()['features']

In [167]:
#Ed

In [168]:
for idx,ii in enumerate(Ed):
    if idx==0:
        df=pd.DataFrame([ii['properties']])
    else:
        df=pd.concat([df,pd.DataFrame([ii['properties']])])        
df=df.reset_index()

for idx,ii in enumerate(Ud):
    if idx==0:
        df1=pd.DataFrame([ii['properties']])
    else:
        df1=pd.concat([df1,pd.DataFrame([ii['properties']])]) 
        
df1=df1.reset_index()
df['uncertainty']=df1['error']     
df.to_excel('C:/Djamai_Najib/1_vegetation_parameters/1_scripts/SL2P_python/testdata/'+variable+'_samples_2.xlsx')  
variable

'CCC'

In [169]:
df

,index,estimate,networkID,partition,uncertainty
0,0,140.765327,0,15,30.340917
1,0,148.508772,0,15,31.368784
2,0,148.508772,0,15,31.368784
3,0,148.508772,0,15,31.368784
4,0,86.530519,0,15,25.516392
...,...,...,...,...,...
1705,0,11.801934,0,16,11.042138
1706,0,11.801934,0,16,11.042138
1707,0,11.801934,0,16,11.042138
1708,0,7.417290,0,15,10.939264


In [170]:
variable

'CCC'

In [153]:
df1

,index,error,networkID,partition
0,0,0.026139,0,15
1,0,0.026080,0,15
2,0,0.026080,0,15
3,0,0.026080,0,15
4,0,0.026864,0,15
...,...,...,...,...
1705,0,0.044153,0,16
1706,0,0.044153,0,16
1707,0,0.044153,0,16
1708,0,0.045263,0,15


In [154]:
estimateSL2P.first().bandNames().getInfo()

['estimate', 'partition', 'networkID']

In [184]:
products = input_collection.select(['date','QC']).combine(estimateSL2P).combine(uncertaintySL2P)

In [185]:
productCollection, outputScale, sampleRegion, sampleName=products, outputScale, site.buffer(bufferSize).geometry(),"sample" + outputName

In [186]:
# returns lists of sampled values for each band in an image as a new feature property
# print('sampleProductCollection')
productCollection = ee.ImageCollection(productCollection)
outputScale= ee.Number(outputScale)
sampleRegion = ee.Feature(sampleRegion)

sampleData = productCollection.map(lambda image: image.sample(region=sampleRegion.geometry(), projection=image.select('date').projection(), scale=outputScale,geometries=True, dropNulls = False) ).flatten()
sampleList= ee.List(productCollection.first().bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)}))) 
sampleFeature=sampleRegion.set(sampleName,sampleList)

In [187]:
R=sampleFeature.getInfo()['properties']['sample'+variable]

In [188]:
df=pd.DataFrame()
for ii in [0,1,2,5]:
    df[R[ii]['bandName']]=R[ii]['data']
df
df.to_excel('C:/Djamai_Najib/1_vegetation_parameters/1_scripts/SL2P_python/testdata/'+variable+'samples.xlsx')    

In [189]:
SL2P_nets=COLLECTION_OPTIONS[collectionName]['Collection_SL2P'].getInfo()
SL2P_error_nets=COLLECTION_OPTIONS[collectionName]['Collection_SL2Perrors'].getInfo()

In [173]:
print([net['id'] for net in SL2P_nets['features']])
print([net['id'] for net in SL2P_error_nets['features']])

['00000000000000000003', '00000000000000000000', '00000000000000000001', '00000000000000000004', '00000000000000000005', '00000000000000000006', '00000000000000000002']
['00000000000000000000', '00000000000000000001', '00000000000000000002', '00000000000000000003', '00000000000000000004', '00000000000000000005', '00000000000000000006']


In [112]:
[sampleFeature.getInfo()['properties']['sampleLAI'][ii]['bandName'] for ii in range(len(sampleFeature.getInfo()['properties']['sampleLAI']))]

['date',
 'QC',
 'estimate',
 'partition',
 'networkID',
 'error',
 'partition_1',
 'networkID_1']